# Blue Brain BioExplorer

![](../../bioexplorer_banner.png)

## Simplest script to create a visualization of a cell

In [ ]:
from bioexplorer import BioExplorer, MovieMaker, Protein, Membrane, Cell, AnimationParams, Vector3, Quaternion
import os

be = BioExplorer('localhost:5000')
core = be.core_api()
status = be.reset_scene()

In [ ]:
def add_cell(random_seed):
    
    core.set_application_parameters(image_stream_fps=0)
    representation=be.REPRESENTATION_ATOMS_AND_STICKS

    # Resources
    resource_folder = '../../../tests/test_files'
    pdb_folder = os.path.join(resource_folder, 'pdb')
    membrane_folder = os.path.join(pdb_folder, 'membrane')

    # ACE2 receptor definition
    ace2_receptor = Protein(
        name=be.NAME_RECEPTOR,
        source=os.path.join(pdb_folder, '6m18.pdb'),
        occurrences=8,
        position=Vector3(0.0, 6.0, 0.0),
        rotation=Quaternion(0.0, 1.0, 0.0, 0.0)
    )

    # Membrane definition
    membrane = Membrane(
        lipid_sources=[
            os.path.join(membrane_folder, 'segA.pdb'),
            os.path.join(membrane_folder, 'segB.pdb'),
            os.path.join(membrane_folder, 'segC.pdb'),
            os.path.join(membrane_folder, 'segD.pdb')],
        animation_params=AnimationParams(random_seed, random_seed + 1, 1.0, random_seed + 2, 0.025)
    )

    # Cell definition
    cell_size = Vector3(800.0, 80.0, 800.0)
    cell = Cell(
        name='Cell', shape_params=cell_size,
        shape=be.ASSEMBLY_SHAPE_SINUSOID,
        membrane=membrane, proteins=[ace2_receptor]
    )

    # Add cell to scene
    be.add_cell(cell=cell, representation=representation)
    
    # Materials
    be.apply_default_color_scheme(be.SHADING_MODE_DIFFUSE)
    
    be.set_models_visibility(True)
    core.set_application_parameters(image_stream_fps=20)

In [ ]:
status = core.set_camera(
    orientation=[-0.085, 0.340, 0.022, 0.936],
    position=[461.372, 55.791, 480.118],
    target=[23.322, -65.160, -40.892]
)

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

In [ ]:
from tqdm import tqdm

mm = MovieMaker(be)
output_folder = '/tmp'

for frame in tqdm(range(1, 20)):
    '''Cell'''
    add_cell(frame * 3)
    
    '''Snapshot'''
    mm.create_snapshot(
        renderer='bio_explorer',
        size=[256, 256], samples_per_pixel=64,
        path=output_folder, base_name='%05d' % frame)